In [1]:
import cv2
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import shutil

from tqdm import tqdm

In [3]:
from tactile_learning.datasets import dump_video_to_images, get_closest_id
from tactile_learning.utils.visualization import plot_tactile_sensor

In [4]:
import decimal

# Get the metadata of images and tactile information
def get_desired_indices(root, fps, view_num=0): # frames per second from the video to receive
    image_metadata_path = os.path.join(root, f'cam_{view_num}_rgb_video.metadata')
    tactile_info_path = os.path.join(root, 'touch_sensor_values.h5')

    with open(image_metadata_path, 'rb') as f:
        image_metadata = pickle.load(f)
        image_timestamps_array = np.asarray(image_metadata['timestamps'])
        image_timestamps = np.asarray(image_metadata['timestamps']) / 1000.
    with h5py.File(tactile_info_path, 'r') as f:
        tactile_timestamps = f['timestamps'][()]

    image_id, tactile_id = 0, 0
    curr_timestamp = tactile_timestamps[0] # These timestamps are in seconds
    image_id = get_closest_id(image_id, curr_timestamp, image_timestamps)

    tactile_indices, image_indices = [], []
    tactile_indices.append(tactile_id)
    image_indices.append(image_id)

    frame_period = 1. / fps
    while(True):
        curr_timestamp += frame_period
        tactile_id = get_closest_id(tactile_id, curr_timestamp, tactile_timestamps)
        image_id = get_closest_id(image_id, curr_timestamp, image_timestamps)

        if curr_timestamp > tactile_timestamps[tactile_id] and curr_timestamp > image_timestamps[image_id]:
            break

        tactile_indices.append(tactile_id)
        image_indices.append(image_id)

    assert len(tactile_indices) == len(image_indices)
    return tactile_indices, image_indices


In [5]:

def dump_states(root, tactile_indices, image_indices, view_num=0):
    # Make directory to dump the visualization
    pbar = tqdm(total=len(tactile_indices))

    with h5py.File(os.path.join(root, 'touch_sensor_values.h5'), 'r') as f:
        all_tactile_values = f['sensor_values'][()]

    viz_dir = os.path.join(root, 'visualization')
    os.makedirs(viz_dir, exist_ok=True)
    
    video_path = os.path.join(root, f'cam_{view_num}_rgb_video.avi')
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_id = 0
    for i in range(len(tactile_indices)):
        tactile_id, image_id = tactile_indices[i], image_indices[i]
        while frame_id != image_id and success:
            # Find the frame that is equal to image_id
            success, image = vidcap.read()
            frame_id += 1
        dump_demo_state(
            frame_id = i,
            viz_dir = viz_dir,
            tactile_values = all_tactile_values[tactile_id,:,:,:],
            camera_img = image
        )

        pbar.update(1)

    pbar.close()

def dump_demo_state(frame_id, viz_dir, tactile_values, camera_img):
    # tactile_values: (15,16,3)
    fig, axs = plt.subplots(figsize=(20,20), nrows=4, ncols=4)
    for row_id in range(4):
        for column_id in range(4):
            if row_id + column_id > 0: # The top left axis should stay empty
                plot_tactile_sensor(
                    ax = axs[column_id][row_id],
                    sensor_values = tactile_values[row_id*4 + column_id-1],
                    title = f'Sensor {row_id*4+column_id-1}'
                )
    fig.suptitle('Tactile State')
    fig.savefig(os.path.join(viz_dir, 'Tactile State.png'))
    fig.clf()
    plt.close()

    tactile_img = cv2.imread(os.path.join(viz_dir, 'Tactile State.png'))
    height_scale = camera_img.shape[0] / tactile_img.shape[0]
    tactile_img = cv2.resize(
        tactile_img,
        (int(tactile_img.shape[1] * height_scale),
         int(tactile_img.shape[0] * height_scale))
    )
    total_img = cv2.hconcat([camera_img, tactile_img])

    img_name = 'state_{}.png'.format(str(frame_id).zfill(3))
    cv2.imwrite(os.path.join(viz_dir, img_name), total_img)



In [8]:
import glob
# roots = [
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/bowl_picking/eval/demonstration_7',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/bowl_picking/eval/demonstration_9',
#     '/home/irmak/Workspace/Holo-Bot/extracted_data/bowl_picking/eval/demonstration_15',
# ]
data_path = '/home/irmak/Workspace/Holo-Bot/extracted_data/cup_picking/after_rss'
# data_path = '/data/tactile_learning/deployment_data/data/bowl_picking/demonstrations/image_generalization'
roots = glob.glob(f'{data_path}/demonstration_*')

retrieval_fps = 1
view_num = 1
for root in roots:
    # print('root: {}'.format(root))
    tactile_indices, image_indices = get_desired_indices(root=root, fps=retrieval_fps, view_num=view_num)
    print('tactile_indices: {}, image_indices: {}'.format(tactile_indices, image_indices))
    dump_states(root, tactile_indices, image_indices, view_num=view_num)


tactile_indices: [0, 99, 197, 295, 394, 491, 590, 689, 786, 884, 982, 1081, 1178, 1276, 1375, 1472, 1571, 1668, 1766, 1866, 1965, 2062, 2160, 2259, 2356, 2455, 2554, 2652, 2750, 2848, 2947, 3045, 3144, 3242, 3341, 3439, 3539, 3637, 3735, 3834, 3938, 4031, 4129, 4227, 4326, 4424, 4523, 4621, 4720, 4818], image_indices: [73, 103, 133, 163, 193, 223, 253, 283, 313, 343, 373, 403, 433, 463, 493, 523, 553, 583, 613, 643, 672, 701, 731, 761, 791, 821, 851, 881, 911, 941, 971, 1001, 1030, 1060, 1090, 1120, 1150, 1180, 1210, 1240, 1270, 1300, 1330, 1360, 1390, 1420, 1450, 1480, 1510, 1540]




  0%|                                                                                          | 0/50 [00:00<?, ?it/s]

  2%|█▋                                                                               | 5/246 [00:15<12:48,  3.19s/it]


  4%|███▎                                                                              | 2/50 [00:02<00:59,  1.25s/it]

  6%|████▉                                                                             | 3/50 [00:03<00:55,  1.17s/it]

  8%|██████▌                                                                           | 4/50 [00:04<00:52,  1.13s/it]

 10%|████████▏                                                                         | 5/50 [00:05<00:54,  1.21s/it]

 12%|█████████▊                                                                        | 6/50 [00:07<00:50,  1.16s/it]

 14%|███████████▍                                                                      | 7/50 [00:08<00:51,  1.19s/it]

 16%|█████████████                   

tactile_indices: [0, 99, 198, 296, 395, 493, 592, 688, 787, 885, 983, 1081, 1180, 1278, 1375, 1474, 1572, 1670, 1769, 1871, 1965, 2064, 2162, 2261, 2359, 2457, 2555, 2652, 2751, 2850, 2947, 3044, 3142, 3240, 3337, 3437, 3534, 3631, 3730, 3828, 3926], image_indices: [70, 100, 130, 160, 190, 220, 250, 280, 310, 340, 369, 398, 428, 458, 488, 518, 548, 578, 608, 638, 668, 698, 728, 758, 787, 817, 847, 877, 907, 937, 966, 996, 1026, 1056, 1085, 1115, 1145, 1175, 1205, 1235, 1264]


100%|█████████████████████████████████████████████████████████████████████████████████| 41/41 [00:46<00:00,  1.13s/it]


tactile_indices: [0, 100, 197, 296, 393, 491, 589, 687, 785, 885, 983, 1080, 1179, 1278, 1375, 1473, 1572, 1669, 1767, 1865, 1963, 2062, 2160, 2259, 2357, 2455, 2553, 2651, 2750, 2849, 2946, 3045, 3144, 3242, 3341, 3439, 3538, 3635], image_indices: [71, 101, 131, 161, 191, 221, 251, 281, 311, 341, 371, 401, 431, 461, 491, 521, 551, 581, 610, 640, 670, 700, 730, 760, 790, 820, 850, 880, 910, 940, 970, 1000, 1030, 1060, 1090, 1120, 1150, 1180]


...

100%|█████████████████████████████████████████████████████████████████████████████████| 50/50 [00:58<00:00,  1.16s/it]


tactile_indices: [0, 99, 198, 295, 392, 491, 590, 687, 785, 883, 981, 1079, 1182, 1274, 1372, 1471, 1567, 1667, 1764, 1863, 1961, 2058, 2157, 2255, 2353, 2452, 2550, 2647, 2746, 2844, 2941, 3040, 3137, 3235, 3334, 3433, 3530, 3629, 3727, 3827, 3925, 4022, 4120, 4219], image_indices: [75, 105, 135, 165, 195, 225, 255, 285, 315, 345, 375, 405, 435, 465, 495, 525, 555, 585, 615, 645, 675, 705, 735, 765, 795, 825, 855, 885, 915, 945, 975, 1005, 1035, 1065, 1095, 1125, 1155, 1185, 1215, 1245, 1275, 1305, 1335, 1365]


100%|█████████████████████████████████████████████████████████████████████████████████| 44/44 [00:50<00:00,  1.15s/it]


tactile_indices: [0, 101, 197, 295, 393, 491, 589, 687, 785, 883, 981, 1080, 1178, 1276, 1374, 1473, 1570, 1667, 1763, 1862, 1960, 2058, 2157, 2254, 2352, 2450, 2548, 2648, 2745, 2843, 2942, 3040, 3139, 3236, 3335, 3432, 3530, 3628], image_indices: [72, 102, 132, 162, 192, 222, 252, 282, 312, 342, 372, 402, 432, 462, 492, 522, 552, 582, 612, 642, 672, 702, 731, 761, 791, 821, 851, 881, 911, 941, 971, 1001, 1031, 1061, 1091, 1121, 1151, 1181]


100%|█████████████████████████████████████████████████████████████████████████████████| 38/38 [00:43<00:00,  1.14s/it]


In [9]:
print(roots)

['/home/irmak/Workspace/Holo-Bot/extracted_data/cup_picking/after_rss/demonstration_10', '/home/irmak/Workspace/Holo-Bot/extracted_data/cup_picking/after_rss/demonstration_8', '/home/irmak/Workspace/Holo-Bot/extracted_data/cup_picking/after_rss/demonstration_3', '/home/irmak/Workspace/Holo-Bot/extracted_data/cup_picking/after_rss/demonstration_1', '/home/irmak/Workspace/Holo-Bot/extracted_data/cup_picking/after_rss/demonstration_2', '/home/irmak/Workspace/Holo-Bot/extracted_data/cup_picking/after_rss/demonstration_9', '/home/irmak/Workspace/Holo-Bot/extracted_data/cup_picking/after_rss/demonstration_6', '/home/irmak/Workspace/Holo-Bot/extracted_data/cup_picking/after_rss/demonstration_7', '/home/irmak/Workspace/Holo-Bot/extracted_data/cup_picking/after_rss/demonstration_5', '/home/irmak/Workspace/Holo-Bot/extracted_data/cup_picking/after_rss/demonstration_4']


In [10]:
# Turn the images to a video and delete the directory
video_fps = 10
for root in roots:
    print('dumping video in root: {}'.format(root))
    video_path = os.path.join(root, f'visualization_{view_num}.mp4')
    if os.path.exists(video_path):
        os.remove(video_path)
    viz_dir = os.path.join(root, 'visualization')
    os.system('ffmpeg -r {} -i {}/%*.png -vf scale=2000x720,setsar=1:1 {}'.format(
        video_fps, # fps
        viz_dir,
        video_path
    ))


dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/cup_picking/after_rss/demonstration_10


ffmpeg version 5.1.2 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 10.4.0 (conda-forge gcc 10.4.0-18)
  configuration: --prefix=/home/irmak/miniconda3/envs/tactile_learning --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1666357487580/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-pthreads --enable-vaapi --enable-gpl --enable-libx264 --enable-libx265 --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-pic --enable-shared --disable-

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/cup_picking/after_rss/demonstration_8


[libx264 @ 0x56042f3d6c00] using SAR=1/1
[libx264 @ 0x56042f3d6c00] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x56042f3d6c00] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x56042f3d6c00] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/cup_picking/after_rss/de

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/cup_picking/after_rss/demonstration_3


...

[libx264 @ 0x555b04090dc0] using SAR=1/1
[libx264 @ 0x555b04090dc0] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x555b04090dc0] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x555b04090dc0] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/cup_picking/after_rss/de

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/cup_picking/after_rss/demonstration_5


[libx264 @ 0x558906cf90c0] using SAR=1/1
[libx264 @ 0x558906cf90c0] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x558906cf90c0] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x558906cf90c0] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/cup_picking/after_rss/de

dumping video in root: /home/irmak/Workspace/Holo-Bot/extracted_data/cup_picking/after_rss/demonstration_4


[libx264 @ 0x55c3f3b39a80] using SAR=1/1
[libx264 @ 0x55c3f3b39a80] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x55c3f3b39a80] profile High 4:4:4 Predictive, level 4.0, 4:4:4, 8-bit
[libx264 @ 0x55c3f3b39a80] 264 - core 164 r3095 baee400 - H.264/MPEG-4 AVC codec - Copyleft 2003-2022 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=22 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=10 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/irmak/Workspace/Holo-Bot/extracted_data/cup_picking/after_rss/de

In [11]:
# Remove the img directory

for root in roots:
    viz_dir = os.path.join(root, 'visualization')
    shutil.rmtree(viz_dir)
